In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv(''C:/Users/HUAWEI/ML/pythonProject/train.csv'')
test_data = pd.read_csv('/Users/konstantin/MachineLearning/task4/data/test.csv')
#delete ids
train_data = train_data.drop(columns=["Id"])
val_ids = test_data["Id"]  #remember for submission
test_data = test_data.drop(columns=["Id"])
# delete nans
for feature in train_data.columns:
    percent = train_data[feature].isnull().sum() / train_data.shape[0]
    if percent > 0.7:
        train_data = train_data.drop(columns=feature)
        test_data = test_data.drop(columns=feature)
# need to convert data
for feature in train_data.columns[:-1]:
    if train_data[feature].dtype == 'object':
        train_data[feature] = LabelEncoder().fit_transform(train_data[feature])
        test_data[feature] = LabelEncoder().fit_transform(test_data[feature])
# split
X_train, X_test, y_train, y_test = train_test_split(train_data.drop(columns='SalePrice').values,
                                                    np.log1p(train_data['SalePrice'].values), test_size=0.2,
                                                    random_state=98987)
X_train = SimpleImputer(strategy='most_frequent').fit_transform(X_train)
X_test = SimpleImputer(strategy='most_frequent').fit_transform(X_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train, y_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'num_leaves': [10, 30, 60, 90],
    'learning_rate': [0.001, 0.005, 0.01],
    'max_depth': [2, 4, 6, 8],
    'n_estimators': [100, 1000, 5000],
    'max_bins': [128, 256, 512], }
lgb_model = GridSearchCV(lgb.LGBMRegressor(), parameters)
lgb_model.fit(X_train, y_train)
print("Best parameters for LGBM is: {}".format(lgb_model.best_params_))

In [ ]:
lgb_model = lgb.LGBMRegressor(**lgb_model.best_params_)
lgb_model.fit(X_train, y_train)
print('Abs error on train data: %1.2f'%metrics.mean_absolute_error(lgb_model.predict(X_train), y_train))
print('Abs error on test data %1.2f:'%metrics.mean_absolute_error(lgb_model.predict(X_test), y_test))